In [1]:
import requests
import ipywidgets as widgets
from IPython.display import display
import json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import time
from code import interact
import pandas as pd
import markdown
from requests import options 

In [2]:
year = str(2016)
season = 'regular_season'

In [ ]:
import re
global paths
paths = 'data/'+year+'_'+season+'.json'
season_widget = widgets.RadioButtons(
    options=['regular_season', 'playoffs'],
    description='Season:',
    disabled=False,
    )
global path_new
def get_game_ids(path_new):
    """returns a list of all game ids inside a json file"""
    game_ids = []
    with open(path_new) as json_file:
        if re.search('regular_season', path_new):
            data = json.load(json_file)
            game_ids = list(data.keys())
        elif re.search('playoffs', path_new):
            data = json.load(json_file)
            game_ids = list(data.keys())
            print(game_ids)
            # check if the game id is not in playoffs json file, then do not add it to the list
            for game_id in game_ids:
                if game_id not in data.keys():
                    game_ids.remove(game_id)
    return game_ids
    
path_new=paths

def on_season_widget_change(change):
    global path_new
    print("Value of season widget is: ",season_widget.value)
    if change['new'] == 'playoffs':
        season = 'playoffs'
        path_new = 'data/'+year+'_playoffs.json'
        id_widget.max = get_game_ids(path_new)[-1]
        id_widget.min = get_game_ids(path_new)[0]
        id_widget.options = get_game_ids(path_new)
    elif change['new'] == 'regular_season':
        season = 'regular_season'
        path_new = 'data/'+year+'_regular_season.json'
        id_widget.max = get_game_ids(path_new)[-1]
        id_widget.min = get_game_ids(path_new)[0]
        id_widget.options = get_game_ids(path_new)
    else:
        print("Error")

id_widget = widgets.SelectionSlider(
        # value=MIN_ID,
        min=get_game_ids(path_new)[0],
        max=get_game_ids(path_new)[-1],
        # step=1,
        options = get_game_ids(path_new),
        description='game_id:',
        continuous_update=False
    )

def display_details(game_id):
    """displays the event start date and time according to selected game_id"""
    with open(path_new) as json_file:
        data = json.load(json_file)
        game = data[str(game_id)]
        a = "Game Start Date: " + game['gameData']['datetime']['dateTime']
        b = "Game ID: " + str(game_id)+"; "+game['gameData']['teams']['home']['abbreviation']+" (home) vs "+game['gameData']['teams']['away']['abbreviation']+" (away)"
        # Display the 2 teams information in a form of a dataframe
        df = pd.DataFrame({'Team': [game['gameData']['teams']['home']['abbreviation'], game['gameData']['teams']['away']['abbreviation']],
                            'Goals': [game['liveData']['linescore']['teams']['home']['goals'], game['liveData']['linescore']['teams']['away']['goals']],
                            'SOG': [game['liveData']['linescore']['teams']['home']['shotsOnGoal'], game['liveData']['linescore']['teams']['away']['shotsOnGoal']],
                            'SO Goals': [game['liveData']['linescore']['teams']['home']['powerPlay'], game['liveData']['linescore']['teams']['away']['powerPlay']],
                            #'SO Attempts': [game['liveData']['linescore']['teams']['home']['powerPlayOpportunities'], game['liveData']['linescore']['teams']['away']['powerPlayOpportunities']]
                            })
        display(a)
        display(b)
        display(df)

season_widget.observe(on_season_widget_change, names='value')
display(season_widget)
print("Value of season widget is: ",season_widget.value)
widgets.interact(display_details, game_id=id_widget)
